In [1]:
!pip install selenium webdriver-manager pandas


In [9]:
pip install capsolver


Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install fake-useragent


In [10]:
pip install selenium webdriver-manager fake-useragent requests


Note: you may need to restart the kernel to use updated packages.


In [16]:
#Muhamed Hisham bin Mohamed Bahurudeen (IS01081947)
#Muhammad Afiq Fikri Bin Ahmad Sabri (IS01082516)

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import capsolver  # Import CapSolver

# Set your CapSolver API key
capsolver.api_key = "YOUR_CAPSOLVER_API_KEY"  # Replace with your actual API key

# Setup Selenium WebDriver
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Initialize Chrome WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Lazada product review page(Sambal Nyet)
url = "https://www.lazada.com.my/products/sambal-nyet-berapi-ori-dari-khairulaming-i2623278934-s11857920384.html"
driver.get(url)
time.sleep(5)  # Allow page to load

reviews_data = []
max_pages = 5  # Number of pages to scrape

# Function to solve CAPTCHA using CapSolver
def solve_captcha():
    print("Solving CAPTCHA...")
    try:
        solution = capsolver.solve({
            "type": "ReCaptchaV2TaskProxyless",  # For reCAPTCHA v2
            "websiteURL": driver.current_url,   # Current page URL
            "websiteKey": "SITE_KEY_HERE"       # Replace with actual site key if known
        })
        print(f"CAPTCHA Solved: {solution}")
        return solution
    except Exception as e:
        print(f"CAPTCHA solving failed: {e}")
        return None

for page in range(1, max_pages + 1):
    print(f"Scraping Page {page}...")

    # Scroll down to ensure all reviews load
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    # Find all review elements on the page
    reviews = driver.find_elements(By.CLASS_NAME, "item")

    if not reviews:
        print("No reviews found. Checking for CAPTCHA...")
        captcha_solution = solve_captcha()
        if captcha_solution:
            time.sleep(5)
            continue
        else:
            break

    for review in reviews:
        try:
            reviewer = review.find_element(By.XPATH, ".//div[@class='middle']/span").text.strip()
        except:
            reviewer = "No Name"

        try:
            date = review.find_element(By.XPATH, ".//span[@class='title right']").text.strip()
        except:
            date = "No Date"

        try:
            content = review.find_element(By.XPATH, ".//div[@class='content']").text.strip()
        except:
            content = "No Review Content"

        reviews_data.append([reviewer, date, content])

    print(f"Page {page} scraped. Total reviews so far: {len(reviews_data)}")

    # Try navigating to the next page
    try:
        pagination_section = driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[2]/div/div')
        next_page_button = pagination_section.find_element(By.XPATH, f".//button[contains(@class, 'next-pagination-item') and text()='{page + 1}']")

        if next_page_button:
            driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
            time.sleep(2)
            driver.execute_script("window.scrollBy(0, -100);")
            time.sleep(1)
            driver.execute_script("arguments[0].click();", next_page_button)
            print(f"Moving to Page {page + 1}")
            WebDriverWait(driver, 20).until(
                EC.staleness_of(reviews[0])
            )
            time.sleep(5)
        else:
            print("Next page button not found. Stopping scrape.")
            break
    except Exception as e:
        print(f"Error navigating to next page: {e}")
        break

# Close WebDriver
driver.quit()

# Save reviews to CSV file
df = pd.DataFrame(reviews_data, columns=["Reviewer Name", "Review Date", "Review Content"])
df.to_csv("lazada_sambal_nyet_reviews.csv", index=False, encoding="utf-8")

print(f"Scraping complete. {len(reviews_data)} reviews saved to lazada_sambal_nyet_reviews.csv")


2025-03-13 15:58:46,244 - INFO - ====== WebDriver manager ======
2025-03-13 15:58:47,950 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-13 15:58:48,175 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-13 15:58:48,413 - INFO - Driver [C:\Users\isham\.wdm\drivers\chromedriver\win64\134.0.6998.88\chromedriver-win32/chromedriver.exe] found in cache


Scraping Page 1...
Page 1 scraped. Total reviews so far: 5
Moving to Page 2
Scraping Page 2...
Page 2 scraped. Total reviews so far: 10
Moving to Page 3
Scraping Page 3...
Page 3 scraped. Total reviews so far: 15
Moving to Page 4
Scraping Page 4...
Page 4 scraped. Total reviews so far: 20
Moving to Page 5
Scraping Page 5...
Page 5 scraped. Total reviews so far: 25
Moving to Page 6
Error navigating to next page: Message: 

Scraping complete. 25 reviews saved to lazada_sambal_nyet_reviews.csv
